In [20]:
try:
    from .objects import  Functions, Compose, Calls, Composer, Juxtapose 
    from .callables import flipped, do, step, starred, excepts, ifnot, ifthen
except Exception as e:
    from objects import  Functions, Compose, Calls, Composer, Juxtapose
    from callables import flipped, do, step, starred, excepts, ifnot, ifthen
    
from collections import OrderedDict
from functools import partial, wraps
from six import PY3
from operator import attrgetter
from toolz.curried import (isiterable, flip, complement, interpose, groupby, merge, reduce, filter, map)
_attribute_ = "__{}{}__".format

In [17]:
__all__ = ['flips', 'stars', 'does', 'maps', 'filters', 'groups', 'reduces']
functions = (flipped, starred, do, map, filter, groupby, reduce)

In [22]:
class Namespaces(Calls):
    namespaces = OrderedDict({'fidget': {}})
    def __getattr__(self, attr):
        for namespace in self.nameaspaces.values():
            if attr in namespace:
                callable = namespace[attr]
                doc = callable.__doc__
                if callable in type(self).__dict__.values():
                    callable = partial(callable, self)
                else:
                    callable = partial(self.__getitem__, callable)
                return PY3 and setattr(callable, '__doc__',  doc) or callable
        raise AttributeError("No attribute {}".format(attr))
        
    def __dir__(self):
        return list(super(Namespaces, self).__dir__()) + list(merge(self.namespaces.values()).keys())

In [23]:
class Factory(Namespaces):
    @property
    def _factory_(self):
          return type(self).__name__.startswith('_') and type(self).__name__.endswith('_')
        
    def __getitem__(self, object=slice(None), *args, **kwargs):
        self = self._factory_ and self.function() or self
        
        if isinstance(object, Factory) and object._factory_:
            object = object()

        return super(Factory, self).__getitem__(object, *args, **kwargs)

In [24]:
class Models(Factory):    
    def __xor__(self, object):
        self, _isinstance = self[:], flip(isinstance)  # noqa: F823
        if not isiterable(object) and isinstance(object, type):
            object = (object,)
        if isiterable(object):
            if all(map(_isinstance(type), object)) and all(map(flip(issubclass)(BaseException), object)):
                self.function = Compose(excepts(object, self.function))
                return self
            
            if all(map(_isinstance(BaseException), object)):
                object = tuple(map(type, object))
                
            if all(map(_isinstance(type), object)):
                object = _isinstance(object)

        self.function = Compose([ifthen(Compose([object]), self.function)])
        return self

    def __or__(self, object):
        self = self[:]
        self.function = Compose([ifnot(self.function, Compose([object]))])
        return self
    
    def __and__(self, object):
        self = self[:]
        self.function = Compose([step(self.function, Compose([object]))])
        return self
    
    def __pos__(self):
        return self[bool]

    def __neg__(self):
        return self[complement(bool)]
        
    def __lshift__(self, object):
        return Does()[object] if self._factory_ else self[do(object)]
    
    def __round__(self, n):
        self.function.function = list(interpose(n, self.function.function))
        return self
    
    __invert__, __pow__ = Functions.__reversed__, __xor__
    __mul__ = __add__ = __rshift__ = __sub__ = Composer.__getitem__

In [21]:
@property
def doc(self):
    string=""
    for function in self:
        string += getattr(function, 'func', function).__doc__ or "" + "\n---\n"
    return string

for klass in (Models, Compose, Juxtapose):
    klass.__doc__ = doc

In [19]:
for name, function in zip(__all__, functions):
    locals().update({name.capitalize(): type(name, (Models,), {'_decorate_': staticmethod(function)})})

In [15]:
__all__ += ['models']

In [6]:
for fidget in __all__:
    callable = locals()[fidget.capitalize()]
    locals()[fidget] = type('_{}_'.format(fidget.capitalize()), (callable,), {})(function=Compose([callable]))

In [8]:
for op, func in (('matmul', 'groupby'), ('truediv', 'map'), ('floordiv', 'filter'), ('mod', 'reduce')):
    setattr(Models, _attribute_('', op), property(Compose(attrgetter(func))))
Models.__div__  = Models.__truediv__ 

In [9]:
def fallback(attr):
    def fallback(right, left):
        right = right[:]
        return getattr(type(right)()[left], attr)(right)
    return wraps(getattr(Models, attr))(fallback)

In [10]:
for attr in ['add', 'sub', 'mul', 'matmul','div', 'truediv', 'floordiv', 'mod', 'lshift', 'rshift', 'and', 'xor', 'or', 'pow']:
    setattr(Models, _attribute_('i', attr), getattr(Models, _attribute_('', attr)))
    setattr(Models, _attribute_('r', attr), fallback(_attribute_('', attr)))